In [1]:
%pip install python-dotenv jupysql duckdb duckdb-engine sqlalchemy openai twitter-api-client --quiet
%load_ext sql

Note: you may need to restart the kernel to use updated packages.


# Elementus API Usage Example

In [2]:
from elementus import ElementusClient, ElementusAPIError
import matplotlib.pyplot as plt
import pandas as pd
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
from helper import *
import os

# Load the environment variables
load_env()

# Retrieve Google BigQuery connection string from the environment
GBQ_URI = get_gbq_uri()
engine = create_engine(GBQ_URI)

# Retrieve the API key from the environment
ELEMENTUS_API_KEY = get_elementus_api_key()

/opt/homebrew/Caskroom/miniforge/base/envs/torch/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


### Retrieve last Bitcoin block transactions over 1 BTC

In [3]:
%sql engine

In [4]:
%%sql result <<
WITH max_block AS (
  SELECT MAX(number) as max_number 
  FROM `bigquery-public-data.crypto_bitcoin.blocks`
),
inputs AS (
  SELECT DISTINCT
    transaction_hash AS tx_hash,
    addresses
  FROM `bigquery-public-data.crypto_bitcoin.inputs`, max_block
  WHERE block_number = max_number
),
outputs AS (
  SELECT 
    transaction_hash AS tx_hash,
    addresses[OFFSET(0)] AS address,
    value / 100000000 AS value
  FROM `bigquery-public-data.crypto_bitcoin.outputs`, max_block
  WHERE block_number = max_number
)
SELECT 
    o.tx_hash, 
    o.address AS output_address, 
    o.value,
    ARRAY_AGG(DISTINCT input_address) AS input_addresses
FROM outputs o
JOIN inputs i ON o.tx_hash = i.tx_hash
CROSS JOIN i.addresses AS input_address
WHERE o.address NOT IN UNNEST(i.addresses) AND 
o.address NOT IN ('bc1qv4r4zwvf5ckdx3llaykl0vez4fdaj4jh5yfjsw') AND 
i.addresses[OFFSET(0)] NOT IN ('bc1qv4r4zwvf5ckdx3llaykl0vez4fdaj4jh5yfjsw') AND 
o.value >= 1
GROUP BY o.tx_hash, o.address, o.value
ORDER BY value DESC

Running query in 'bigquery://elementus-prod-242016'

4 rows affected.

In [5]:
df = result.DataFrame()
df

,tx_hash,output_address,value,input_addresses
0,519e7ab93eb7558c2accc7bee577eb6b1278e48c0b311b...,33kbs53jFRPeJvQun53CDZFRSfgMEGMcuq,12.23426016,"[3AuCaiFowsq1YzCsHxhFXCPs8dBNXsaqnu, 3PJW588RH..."
1,cbb3315df8077a77caaca961338a8f2f0d7d2cd1ff0667...,bc1qqrp5clgngwlalsqnkenpn65ea3skg3rtp0ft39,3.43644584,[bc1qc32jkh35ppucz2fdqkknsq4tjyr0t7w2t80dkc]
2,519e7ab93eb7558c2accc7bee577eb6b1278e48c0b311b...,3PJW588RH2C2YTXTZf122jAwFNKbvotXrW,2.97012965,[3AuCaiFowsq1YzCsHxhFXCPs8dBNXsaqnu]
3,9ea7a1c35677c86ba3882b98965e1ea226e3edd38960aa...,bc1pznwj2846dxwceh24xfpqnrtmaxcum9fdz67mrkmrw4...,1.7047335,[bc1paa5xu7h4lln4fj44hkhpa3h85jqznxan5a9j3rcd9...


### Use Elementus API to retrieve entity attributions for input and output addresses

In [6]:
client = ElementusClient(api_key=ELEMENTUS_API_KEY)

# Check API health
is_healthy = await client.check_health()

output_addresses = df['output_address'].unique().tolist()
input_addresses = list(set(addr for sublist in df['input_addresses'] for addr in sublist))

try:
    response = await client.get_address_attributions(output_addresses)
except ElementusAPIError as e:
    print(f"API error: {e}")

# Create a dictionary from the response data
attribution_dict = {address: attr.entity for address, attr in response.data.items()}

try:
    response = await client.get_address_attributions(input_addresses)
except ElementusAPIError as e:
    print(f"API error: {e}")

attribution_dict.update({address: attr.entity for address, attr in response.data.items()})

len(attribution_dict)

3

### Add entity attributions to transaction data

In [7]:
df['output_entity'] = df['output_address'].map(attribution_dict)
df['input_entity'] = [
    next((attribution_dict[key] for key in addresses if key in attribution_dict), None)
    for addresses in df['input_addresses']
]

df.dropna(subset=['input_entity', 'output_entity'], how='all', inplace=True)
df = df[df['input_entity'] != df['output_entity']]
# Ensure 'input_entity' is a string type before using str.startswith
df = df[~df['input_entity'].astype(str).str.startswith('fastmoney_') & ~df['output_entity'].astype(str).str.startswith('fastmoney_')]

df

,tx_hash,output_address,value,input_addresses,output_entity,input_entity
0,519e7ab93eb7558c2accc7bee577eb6b1278e48c0b311b...,33kbs53jFRPeJvQun53CDZFRSfgMEGMcuq,12.23426016,"[3AuCaiFowsq1YzCsHxhFXCPs8dBNXsaqnu, 3PJW588RH...",okx.com,nexo.io


In [8]:
# Convert DataFrame to JSON string
json_string = df[['tx_hash', 'input_entity', 'output_entity', 'value']].to_json(orient='records')

json_string

'[{"tx_hash":"519e7ab93eb7558c2accc7bee577eb6b1278e48c0b311bb9c7de9cc3779748f5","input_entity":"nexo.io","output_entity":"okx.com","value":12.23426016}]'

### Create X Post using X AI to generate on-chain insights from transaction data

In [9]:
from openai import OpenAI
from IPython.display import display, Markdown

XAI_API_KEY = get_xai_api_key()

client = OpenAI(
    api_key=XAI_API_KEY,
    base_url="https://api.x.ai/v1",
)

completion = client.chat.completions.create(
    model="grok-2-latest",
    messages=[
        {
            "role": "system",
            "content": "You are an expert in Bitcoin blockchain analytics and market structure"
        },
        {
            "role": "user",
            "content": "Given a list of Bitcoin transactions attributed to entities and news currently trending on Crypto Twitter create an X post explaining the most inportant on-chain insights, which may affect BTC/USD price. Provide only the text ready to be sent via X API without any additional comments. Limit the text to 160 characters. " + json_string
        },
    ],
    temperature=0,
)

display(Markdown(f"## X Post\n{completion.choices[0].message.content}"))


## X Post
On-chain data shows a significant transfer of 12.23 BTC from Nexo to OKX. This could signal increased liquidity and impact BTC/USD price. #Bitcoin #CryptoNews

### Tweet the generated post

In [10]:
from twitter.account import Account

account = Account(cookies={"ct0": "", "auth_token": ""})
account.tweet(text=completion.choices[0].message.content)

{'data': {'create_tweet': {'tweet_results': {'result': {'rest_id': '1890860377707073623',
     'has_birdwatch_notes': False,
     'core': {'user_results': {'result': {'__typename': 'User',
        'id': 'VXNlcjo5MDA3MjgwNTgyNTYxMzgyNDA=',
        'rest_id': '900728058256138240',
        'affiliates_highlighted_label': {},
        'has_graduated_access': True,
        'is_blue_verified': True,
        'profile_image_shape': 'Circle',
        'legacy': {'can_dm': True,
         'can_media_tag': True,
         'created_at': 'Thu Aug 24 14:34:38 +0000 2017',
         'default_profile': False,
         'default_profile_image': False,
         'description': 'We enable companies of all sizes to make better, faster decisions on the blockchain',
         'entities': {'description': {'urls': []},
          'url': {'urls': [{'display_url': 'elementus.io',
             'expanded_url': 'https://elementus.io/',
             'url': 'https://t.co/h7wrEp5WjF',
             'indices': [0, 23]}]}},
    